In [1]:
import glob
import pandas as pd

from os.path import basename
import pickle

from level2.level2_utils import *
from level2.level2_roi_extraction import level2_roi_extractor
from level2.level2_roi_extraction import load_rois, get_roi_data_for_beta
from level2.level2_roi_extraction import get_roi_data_for_l2_betas, get_roi_data_for_multirun_l2_betas
#import modules from files in a parallel directory "direct_regression"


from direct_regression.get_all_series import get_beta_img, get_roi_data, get_moment_trial_type_revealed, get_behavioral_data_with_moment_trial_type_revealed, mask_3d_subject_image
from direct_regression.get_all_series import get_all_subj_df
from direct_regression.fmri_utils import *



In [2]:

config = load_config("direct_regression/config.yml")
analysis_name = 'conditions'


#get the masks
mask_locations = [
    config['mask_location'] + 'aim3/masks/neurosynth/',
    config['mask_location'] + 'aim3/masks/'
]

signature_locations = [
    config['mask_location'] + 'signatures/aim3/'
]

roi_df = get_mask_df_from_mask_locations(mask_locations)
signature_df = get_mask_df_from_mask_locations(signature_locations)
roi_df['image_type'] = 'mask'
signature_df['image_type'] = 'signature'

#combine the two dfs
roi_df = pd.concat([roi_df, signature_df])
#get the list of raw nii files
glob_path = config['fmriprep_dir'] + config['nii_raw_path']

Benjamins-MacBook-Pro-2.local
{'dev_scripts_path': '/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/', 'level2_output_path': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/level2/SST/', 'rural_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/', 'rubric_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/DEV_scoring_rubrics', 'l2_analysis_files': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/level2/', 'input_mat_path': '/Users/benjaminsmith/oregon/code/DEV_scripts/fMRI/fx/multiconds/SST/full_duration/conditions', 'dropbox_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/', 'dev_analysis_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/ana

## SST

In [ ]:
#filter the mask_label in mask_df, using regex, to only use stiraum, finger movements, motor control, and response inbhitioin
sst_roi_df = roi_df.loc[roi_df['mask_label'].str.contains('striatum|finger|motor|response inhibition')]

## SST
train_betas_with_data = get_data_for_confirmed_train_subjs(
    beta_glob = config['nonbids_data_path'] + "fMRI/fx/models/SST/wave1/conditions/sub-DEV*/",
    nonbids_data_path = config['nonbids_data_path'],
    #ml_data_folderpath = ml_data_folderpath,
    ml_scripting_path = config['dev_scripts_path'] + "/fMRI/ml",
    dropbox_datapath=config['dropbox_data_dir'],
    exclude_test_subjs=False
)
train_betas_with_data['wave']=1

#we're not interestd in getting contrasts; comment this out.
#betas_with_contrasts = get_contrasts_for_betas(train_betas_with_data)
betas_with_paths = get_beta_fnames_for_beta_dirs(train_betas_with_data[0:5])


In [ ]:

beta_name_list = [
    'CorrectGo',
    'CorrectStop',
    'FailedStop',
    #'Cue',
    #'FailedGo'
    ]

#get the ROI data
roi_data_sst_conditions = get_roi_data_for_l2_betas(betas_with_paths, beta_name_list, sst_roi_df)



In [ ]:

## SST
train_betas_with_data = get_data_for_confirmed_train_subjs(
    beta_glob = config['nonbids_data_path'] + "fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV*/",
    nonbids_data_path = config['nonbids_data_path'],
    #ml_data_folderpath = ml_data_folderpath,
    ml_scripting_path = config['dev_scripts_path'] + "/fMRI/ml",
    dropbox_datapath=config['dropbox_data_dir'],
    exclude_test_subjs=False
)
train_betas_with_data['wave']=1

#we're not interestd in getting contrasts; comment this out.
#betas_with_contrasts = get_contrasts_for_betas(train_betas_with_data)
betas_with_paths = get_beta_fnames_for_beta_dirs(train_betas_with_data[0:3])

beta_name_list = ['CorrectGoFollowingFailedStop']

#get the ROI data
roi_data_sst_posterror_conditions = get_roi_data_for_l2_betas(betas_with_paths, beta_name_list, sst_roi_df)



In [ ]:
roi_data_sst_posterror_conditions

In [ ]:
roi_data_sst.to_csv(config['dropbox_data_dir'] + '/subject_sst_avg_roi_data_raw.csv')

### W1 W2 analysis

In [3]:
#get the masks
mask_locations = [
    config['mask_location'] + 'aim3/masks/neurosynth/',
    config['mask_location'] + 'aim3/masks/',
    config['mask_location'] + 'striatum/',
    config['mask_location']
]
wave_compare_roi_df = get_mask_df_from_mask_locations(mask_locations)

In [4]:
mask_locations

['/Users/benjaminsmith/Google Drive/oregon/data/DEV/brainmaps/aim3/masks/neurosynth/',
 '/Users/benjaminsmith/Google Drive/oregon/data/DEV/brainmaps/aim3/masks/',
 '/Users/benjaminsmith/Google Drive/oregon/data/DEV/brainmaps/striatum/',
 '/Users/benjaminsmith/Google Drive/oregon/data/DEV/brainmaps/']

In [5]:
wave_compare_roi_df

,mask_label,mask_path
0,reward_association-test_z_FDR_0.01,/Users/benjaminsmith/Google Drive/oregon/data/...
1,response inhibition_association-test_z_FDR_0.01,/Users/benjaminsmith/Google Drive/oregon/data/...
2,finger movements_association-test_z_FDR_0.01,/Users/benjaminsmith/Google Drive/oregon/data/...
3,reappraisal_association-test_z_FDR_0.01,/Users/benjaminsmith/Google Drive/oregon/data/...
4,abstract_association-test_z_FDR_0.01,/Users/benjaminsmith/Google Drive/oregon/data/...
5,attention network_association-test_z_FDR_0.01,/Users/benjaminsmith/Google Drive/oregon/data/...
6,motor control_association-test_z_FDR_0.01,/Users/benjaminsmith/Google Drive/oregon/data/...
7,value_association-test_z_FDR_0.01,/Users/benjaminsmith/Google Drive/oregon/data/...
8,striatum_joint_mask,/Users/benjaminsmith/Google Drive/oregon/data/...
9,motor_control_striatum_joint_mask,/Users/benjaminsmith/Google Drive/oregon/data/...


In [6]:
#filter the mask_label in mask_df, using regex, to only use stiraum, finger movements, motor control, and response inbhitioin
sst_roi_df = wave_compare_roi_df.loc[wave_compare_roi_df['mask_label'].str.contains('response inhibition|striatum_joint_reward|value')]


In [7]:

## SST
train_betas_with_data = get_data_for_confirmed_train_subjs(
    beta_glob = config['nonbids_data_path'] + "fMRI/fx/models/SST/all_waves/health_conditions/sub-DEV*/",
    nonbids_data_path = config['nonbids_data_path'],
    #ml_data_folderpath = ml_data_folderpath,
    ml_scripting_path = config['dev_scripts_path'] + "/fMRI/ml",
    dropbox_datapath=config['dropbox_data_dir'],
    exclude_test_subjs=False
)

train_betas_with_data['wave']='all'

loaded 275 rows from data_by_ppt.csv
255 subjects with no reported problems in task Redcap scanner notes, added to the provision useable_dev_id list.
7 subjects excludeable for missing scan data.
251 subjects remaining on the provision useable_dev_id list from the redcap list after excluding subjects with missing scan data.
217 subjects remaining on the provisional useable_dev_id list from the redcap list after excluding subjects excluded by motion quality process.
beta paths before exclusion: 250
beta paths after exclusion via nsc_subject_exclusions and the provision useable_dev_id list: 185


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/fMRI/fx/models/SST/level2/level2_utils.py:258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useable_betas_with_data.sort_values('subject_id', inplace=True)
/var/folders/cj/4mb6t1f906j397tj71pxfxz00000gn/T/ipykernel_13206/377843695.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_betas_with_data['wave']='all'


In [8]:

#we're not interestd in getting contrasts; comment this out.
#betas_with_contrasts = get_contrasts_for_betas(train_betas_with_data[0:5])
#betas_with_paths = get_beta_fnames_for_beta_dirs(train_betas_with_data[0:5])

In [9]:
contrasts_with_paths = get_contrasts_for_betas(train_betas_with_data[0:5])

/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/fMRI/fx/models/SST/level2/level2_utils.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  betas['spm_l2_targeted_contrast_filepath'] = None


/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/SST/all_waves/health_conditions/sub-DEV008/
0: Healthy_Go spmT_0001.nii
1: Healthy_NoGo spmT_0002.nii
2: Unhealthy_Go spmT_0003.nii
3: Unhealthy_NoGo spmT_0004.nii
4: Null_Cue spmT_0005.nii
5: Stop(Healthy>Unhealthy) spmT_0006.nii
6: Stop(Unhealthy>Healthy) spmT_0007.nii
7: Go(Healthy>Unhealthy) spmT_0008.nii
8: Go(Unhealthy>Healthy) spmT_0009.nii
9: Healthy_Go(W2-W1) spmT_0010.nii
10: Healthy_Go(W1-W2) spmT_0011.nii
11: Healthy_NoGo(W2-W1) spmT_0012.nii
12: Healthy_NoGo(W1-W2) spmT_0013.nii
13: Unhealthy_Go(W2-W1) spmT_0014.nii
14: Unhealthy_Go(W1-W2) spmT_0015.nii
15: Unhealthy_NoGo(W2-W1) spmT_0016.nii
16: Unhealthy_NoGo(W1-W2) spmT_0017.nii
17: Null_Cue(W2-W1) spmT_0018.nii
18: Null_Cue(W1-W2) spmT_0019.nii
19: Stop(Healthy>Unhealthy)(W2-W1) spmT_0020.nii
20: Stop(Healthy>Unhealthy)(W1-W2) spmT_0021.nii


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/fMRI/fx/models/SST/level2/level2_utils.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  betas.loc[beta_i, 'contrast_' + contrast['name'] + "_fname"] =contrast['Vspm']['fname']
/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/fMRI/fx/models/SST/level2/level2_utils.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  betas.loc[beta_i, 'contrast_' + contrast['name'] + "_fname"] =contrast['Vspm']['fname']
/Users/benjaminsmith/Google Drive/or

/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/SST/all_waves/health_conditions/sub-DEV013/
0: Healthy_Go spmT_0001.nii
1: Healthy_NoGo spmT_0002.nii
2: Unhealthy_Go spmT_0003.nii
3: Unhealthy_NoGo spmT_0004.nii
4: Null_Cue spmT_0005.nii
5: Stop(Healthy>Unhealthy) spmT_0006.nii
6: Stop(Unhealthy>Healthy) spmT_0007.nii
7: Go(Healthy>Unhealthy) spmT_0008.nii
8: Go(Unhealthy>Healthy) spmT_0009.nii
9: Healthy_Go(W2-W1) spmT_0010.nii
10: Healthy_Go(W1-W2) spmT_0011.nii
11: Healthy_NoGo(W2-W1) spmT_0012.nii
12: Healthy_NoGo(W1-W2) spmT_0013.nii
13: Unhealthy_Go(W2-W1) spmT_0014.nii
14: Unhealthy_Go(W1-W2) spmT_0015.nii
15: Unhealthy_NoGo(W2-W1) spmT_0016.nii
16: Unhealthy_NoGo(W1-W2) spmT_0017.nii
17: Null_Cue(W2-W1) spmT_0018.nii
18: Null_Cue(W1-W2) spmT_0019.nii
19: Stop(Healthy>Unhealthy)(W2-W1) spmT_0020.nii
20: Stop(Healthy>Unhealthy)(W1-W2) spmT_0021.nii
/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/SST/all_waves/health_cond

In [20]:
#from level2.level2_roi_extraction import level2_roi_extractor

contrast_name_list = [
    'Unhealthy_NoGo(W2-W1)',
    'Unhealthy_Go(W2-W1)',
    'Stop(Healthy>Unhealthy)(W2-W1)',
    ]

l2_roi_extractor = level2_roi_extractor(center_data=True,scale_data=True)
l2_roi_extractor.load_all_images_simultaneously=True

#get the ROI data
roi_data_sst_conditions = l2_roi_extractor.get_roi_data_for_l2_contrasts(contrasts_with_paths, contrast_name_list, sst_roi_df)



response inhibition_association-test_z_FDR_0.01
producing the vector for this mask/series...
value_association-test_z_FDR_0.01
producing the vector for this mask/series...
striatum_joint_reward_mask
producing the vector for this mask/series...
value_association_frontal_medial_cortex
producing the vector for this mask/series...
Unhealthy_NoGo(W2-W1)
loading 5images...
...loaded.
0
mean is 0.06116048127866569
std is 0.5463909453756108
raw mean is -0.1297473061435247
raw std is 0.4097005569983938
DEV008, all, Unhealthy_NoGo(W2-W1), response inhibition_association-test_z_FDR_0.01


/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/image.py:1041: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return new_img_like(niimg, result, niimg.affine)


proportion of image active: 0.01383226975097848
activity scalar is 3.0186625e-10
DEV008, all, Unhealthy_NoGo(W2-W1), value_association-test_z_FDR_0.01


/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/image.py:1041: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return new_img_like(niimg, result, niimg.affine)


proportion of image active: 0.016650108360635286
activity scalar is -2.6467427e-11
DEV008, all, Unhealthy_NoGo(W2-W1), striatum_joint_reward_mask


...

/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/image.py:1041: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return new_img_like(niimg, result, niimg.affine)
/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/resampling.py:545: UserWarning: Casting data from int32 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))


proportion of image active: 0.005345483279191523
activity scalar is 5.070105e-10
DEV013, all, Unhealthy_NoGo(W2-W1), value_association_frontal_medial_cortex


/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/resampling.py:274: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn("Resampling binary images with continuous or "


proportion of image active: 0.0007864810288022106
activity scalar is 1.1486676e-09
2


/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/image.py:1041: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return new_img_like(niimg, result, niimg.affine)


In [ ]:
roi_data_sst_conditions.to_csv(config['dropbox_data_dir'] + '/subject_sst_health_avg_roi_data_raw.csv')

## WTP

In [ ]:
#filter the mask_label in mask_df, using regex, to only use stiraum, finger movements, motor control, and response inbhitioin
wtp_mask_df = roi_df.loc[roi_df['mask_label'].str.contains('striatum|value')]


## WTP
train_betas_with_data = get_data_for_confirmed_train_subjs(
    beta_glob = config['nonbids_data_path'] + "fMRI/fx/models/WTP/wave1/conditions_liked_disliked/sub-DEV*/",
    nonbids_data_path = config['nonbids_data_path'],
    #ml_data_folderpath = ml_data_folderpath,
    ml_scripting_path = config['dev_scripts_path'] + "/fMRI/ml",
    dropbox_datapath=config['dropbox_data_dir'],
    exclude_test_subjs=False,
    task='WTP'
).copy()
train_betas_with_data['wave']=1



#we're not interestd in getting contrasts; comment this out.
#betas_with_contrasts = get_contrasts_for_betas(train_betas_with_data)
betas_with_paths = get_beta_fname_list_for_beta_dirs(train_betas_with_data[0:5])

#from level2.level2_roi_extraction import get_roi_data_for_l2_betas, get_roi_data_for_multirun_l2_betas

beta_name_list = [
    'liked'
    ]

#get the ROI data
roi_data_wtp = get_roi_data_for_multirun_l2_betas(betas_with_paths, condition_list = beta_name_list, mask_df = wtp_mask_df)

roi_data_wtp.to_csv(config['dropbox_data_dir'] + '/subject_wtp_avg_roi_data_raw.csv')


## ROC

In [ ]:
roi_df

In [ ]:
roc_mask_df = roi_df.loc[roi_df['mask_label'].str.contains('reappraisal|abstract|attention|striatum|craving|regulate')]


train_betas_with_data = get_data_for_confirmed_train_subjs(
    beta_glob = config['nonbids_data_path'] + "fMRI/fx/models/ROC/wave1/conditions/sub-DEV*/",
    nonbids_data_path = config['nonbids_data_path'],
    #ml_data_folderpath = ml_data_folderpath,
    ml_scripting_path = config['dev_scripts_path'] + "/fMRI/ml",
    dropbox_datapath=config['dropbox_data_dir'],
    exclude_test_subjs=False,
    task='ROC'
).copy()
train_betas_with_data['wave']=1



#we're not interestd in getting contrasts; comment this out.
#betas_with_contrasts = get_contrasts_for_betas(train_betas_with_data)
betas_with_paths = get_beta_fname_list_for_beta_dirs(train_betas_with_data[0:5])



#from level2.level2_roi_extraction import get_roi_data_for_l2_betas, get_roi_data_for_multirun_l2_betas

beta_name_list = [
    'lookCrave',
    'reappraiseCrave'
    ]

#get the ROI data
roi_data_roc = get_roi_data_for_multirun_l2_betas(betas_with_paths, condition_list = beta_name_list, mask_df = roc_mask_df)

roi_data_roc.to_csv(config['dropbox_data_dir'] + '/subject_roc_avg_roi_data_raw.csv')


In [ ]:
#save
with open(config['dropbox_data_dir'] + '/subject_sst_avg_roi_data_raw.pkl', 'wb') as handle:
    pickle.dump(roi_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

